In [227]:
import pandas as pd
import numpy as np
from basketball_reference_web_scraper import client
from basketball_reference_web_scraper.data import Location 
from basketball_reference_web_scraper.data import Team
from basketball_reference_web_scraper.data import Outcome 
from basketball_reference_web_scraper.data import OutputType 
from basketball_reference_web_scraper.data import OutputWriteOption 
from basketball_reference_web_scraper.data import Position
import time
from datetime import datetime, timedelta
# from myPricePicksMethods import *

In [291]:
#players total for this season
df = pd.DataFrame(client.players_season_totals(season_end_year=2024))

In [307]:
df.head()

,slug,name,positions,age,team,games_played,games_started,minutes_played,made_field_goals,attempted_field_goals,...,made_free_throws,attempted_free_throws,offensive_rebounds,defensive_rebounds,assists,steals,blocks,turnovers,personal_fouls,points
0,achiupr01,Precious Achiuwa,[Position.CENTER],24,Team.TORONTO_RAPTORS,25,0,437,78,170,...,24,42,50,86,44,16,12,29,40,193
1,achiupr01,Precious Achiuwa,[Position.POWER_FORWARD],24,Team.NEW_YORK_KNICKS,36,18,962,130,246,...,37,55,115,160,42,26,46,43,78,308
2,adebaba01,Bam Adebayo,[Position.CENTER],26,Team.MIAMI_HEAT,58,58,2002,444,859,...,258,342,135,479,230,63,56,137,137,1152
3,agbajoc01,Ochai Agbaji,[Position.SHOOTING_GUARD],23,Team.UTAH_JAZZ,51,10,1003,106,249,...,15,20,35,91,47,27,29,34,66,274
4,agbajoc01,Ochai Agbaji,[Position.SHOOTING_GUARD],23,Team.TORONTO_RAPTORS,17,9,355,43,113,...,10,15,20,26,16,10,6,17,32,108


In [284]:
#Dont really have to use this you can just use client.regular_season_player_box_scores to get the same info

# all_data_2023 = pd.DataFrame()

# start_date = datetime(2023,10,24)
# end_date = datetime.now()

# currrent_date = start_date
# while currrent_date <= end_date:
#     try:
#         df = pd.DataFrame(client.player_box_scores(day=currrent_date.day, month=currrent_date.month,year=currrent_date.year))
#         all_data_2023 = pd.concat([all_data_2023,df], ignore_index=True)
#     except Exception as e:
#         print(f"Error fetching data for {currrent_date}: {e}")
        
#     currrent_date += timedelta(days=1)
#     time.sleep(5)

In [326]:
#allows me to look up the players slug with his name
def find_slug_by_name(name_dict,player_name):
    for slug, name in name_dict.items():
        if name == player_name:
            return slug
    return None

#allows me to gather players info
def get_players_info(player_name,year):
    return pd.DataFrame(client.regular_season_player_box_scores(
        player_identifier=player_name, 
        season_end_year=year
    ))

#used the get the average of a player for a certain category
def get_players_average_stat(all_data, player_name, stat):
    if stat not in all_data.columns:
        print(f"Stat '{stat}' not found in the data.")
        return None
    
    player_data = all_data[all_data['name'] == player_name]

    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    
    return player_data[stat].mean()

#used to calculate the amount of times the player has hit the under on the projected value
def count_projected(player_name, year, stat, projected_value,bet='over'):
    player_data = get_players_info(player_name,year)

    proj_value = np.ceil(projected_value) if bet == 'over' else np.floor(projected_value)
    
    if stat not in player_data.columns:
        print(f"Stat '{stat}' not found in the data. ")
        return None

    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    if bet == 'over':    
        count = (player_data[stat] >= proj_value).sum()
    else:
        count = (player_data[stat] <= proj_value).sum()

    games_played = player_data.shape[0]
    percentage = (count/games_played).round(2)

    print(f"{name_dict[player_name]} has gone {bet} on {projected_value} in {columns_rename[stat]} a total of {count} times in {games_played} games played which is {percentage}%.")
    return count

#used to calculate the amount of times they've hit the under in most recent games
def count_projected_within_x_games(player_name, year, stat, projected_value, num_games=None,bet='over'):
    player_data = get_players_info(player_name,year)

    if num_games is not None and num_games < len(player_data):
        player_data = player_data.tail(num_games)

    proj_value = np.ceil(projected_value) if bet =='over' else np.floor(projected_value)
    
    if stat not in player_data.columns:
        print(f"Stat '{stat}' not found in the data. ")
        return None

    if player_data.empty:
        print(f"No data found for player: {player_name}")
        return None
    
    if bet == 'under':
        count = (player_data[stat] >= proj_value).sum() 
    elif bet == 'over':
        count = (player_data[stat] >= proj_value).sum()

    games_played = player_data.shape[0]
    percentage = (count/num_games).round(2)

    print(f"{name_dict[player_name]} has gone {bet} on {projected_value} in {columns_rename[stat]} a total of {count} times in his last {num_games} games played which is {percentage}%.")
    return count


In [327]:
#import slug name into a dictionary with name as value 

name_dict = {}

for index,row in df.iterrows():
    slug = row['slug']
    name = row['name']
    if slug not in name_dict:
        if name not in name_dict.values():
            name_dict[slug] = name
        else:
            print(f"Player name {name} already exists in the dictionary with a different slug.")
    else:
        print(f"Slug {slug} already exists in the dictionary with name {name_dict[slug]}")

columns_rename = {
    'games_played': 'games played',
    'made_field_goals': 'made field goals',
    'attempted_field_goals': 'attempted field goals',
    'made_three_point_field_goals': 'made three point field goals',
    'attempted_three_point_field_goals': 'attempted three point field goals',
    'made_free_throws': 'made free throws',
    'attempted_free_throws': 'attempted free throws',
    'offensive_rebounds': 'offensive rebounds',
    'defensive_rebounds': 'defensive rebounds',
    'personal_fouls': 'personal fouls',
    'assists': 'assists',
    'steals':'steals',
    'blocks':'blocks',
    'turnovers':'turnovers',
    'points': 'points'

}

Slug achiupr01 already exists in the dictionary with name Precious Achiuwa
Slug agbajoc01 already exists in the dictionary with name Ochai Agbaji
Slug anunoog01 already exists in the dictionary with name OG Anunoby
Slug baglema01 already exists in the dictionary with name Marvin Bagley III
Slug bantoda01 already exists in the dictionary with name Dalano Banton
Slug barrerj01 already exists in the dictionary with name RJ Barrett
Slug bateske01 already exists in the dictionary with name Keita Bates-Diop
Slug batumni01 already exists in the dictionary with name Nicolas Batum
Slug bertada01 already exists in the dictionary with name Dāvis Bertāns
Slug beverpa01 already exists in the dictionary with name Patrick Beverley
Slug biyombi01 already exists in the dictionary with name Bismack Biyombo
Slug bogdabo02 already exists in the dictionary with name Bojan Bogdanović
Slug boldema01 already exists in the dictionary with name Marques Bolden
Slug brownbr01 already exists in the dictionary with

In [328]:
print(f"The size of the dictionary is {len(name_dict)}.")

The size of the dictionary is 560.


In [329]:
columns_rename = {
    'games_played': 'games played',
    'made_field_goals': 'made field goals',
    'attempted_field_goals': 'attempted field goals',
    'made_three_point_field_goals': 'made three point field goals',
    'attempted_three_point_field_goals': 'attempted three point field goals',
    'made_free_throws': 'made free throws',
    'attempted_free_throws': 'attempted free throws',
    'offensive_rebounds': 'offensive rebounds',
    'defensive_rebounds': 'defensive rebounds',
    'personal_fouls': 'personal fouls',
    'assists': 'assists',
    'steals':'steals',
    'blocks':'blocks',
    'turnovers':'turnovers',
    'points_scored': 'points',
    'game_score': 'game score',
    'plus_minus': 'plus minus'

}

In [347]:
count_projected_within_x_games('davisan02', 2024, 'blocks', 1.5, num_games=15,bet='over')

Anthony Davis has gone over on 1.5 in blocks a total of 10 times in his last 15 games played which is 0.67%.


10

In [349]:
count_projected('davisan02', 2024, 'blocks', 1.5, bet='under')

Anthony Davis has gone under on 1.5 in blocks a total of 23 times in 65 games played which is 0.35%.


23

In [339]:
find_slug_by_name(name_dict,'Anthony Davis')

'davisan02'

In [343]:
name = get_players_info('davisan02', 2024)